In [ ]:
#Check Python Version
import shutil
import sys
import os
sys.version


#Upgrade pip
%pip install --upgrade pip


#Check TF & TFX Versioning
import tensorflow as tf
print(tf.__version__)
from tfx import v1 as tfx
print(tfx.__version__)


# Pipeline name
PIPELINE_NAME = "copy_tfrecords"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = './artifacts'
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

# Folder path to data
DATA_ROOT = './data/'

TFRECORDS_TRAIN_DATA_PATH = '../../../../Documents/split_train/'
TFRECORDS_EVAL_DATA_PATH = '../../../../Documents/split_eval/'
TFRECORDS_GOLDEN_DATA_PATH = '../../../../Documents/split_golden/'

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

Note: you may need to restart the kernel to use updated packages.


2023-03-14 19:16:11.266499: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-14 19:16:11.452554: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-14 19:16:11.452589: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-14 19:16:12.798425: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

2.11.0
1.12.0


In [ ]:
from typing import Dict, List
from tfx.v1.types.standard_artifacts import Examples
from tfx.types import artifact_utils
from typing import Dict

from tfx.dsl.component.experimental.decorators import component

from tfx.dsl.io import fileio

def _split_names_string_builder(split_names_list: List):
    str1 = "["
    urlist_len = len(split_names_list)-1
    index = 0

    for ele in split_names_list:
        if(index==urlist_len):
            str1 += "\""+ele+"\""+"]"
            break
        str1 += "\""+ele+"\""+","
        index+=1
    return str1

@component
def CopyExampleGen(
        input_dict: tfx.dsl.components.Parameter[Dict[str, str]],
        output_example: tfx.dsl.components.OutputArtifact[Examples]
      ) -> tfx.dsl.components.OutputDict():
    
        """Parse input_dict: creates a directory from the split-names and tfrecord uris provided"""
        split_names=[]
        for key in input_dict:
          split_names.append(key)
          
        split_names_string=_split_names_string_builder(split_names)
        output_example.split_names=str(split_names_string)
        
        """Make directories"""
        tfrecords_list=[]
        for key in input_dict:
          split_value="/Split-"+key+"/"
          fileio.mkdir(output_example.uri+split_value)
          tfrecords_list=fileio.glob(input_dict[key]+'*.gz')
          print("tfrecords_list: " + str(tfrecords_list))

          """"Copy files into directories"""
          for tfrecord in tfrecords_list:
                """TODO: Find a better way to extra file name"""
                file_name=os.path.basename(os.path.normpath(tfrecord))
                fileio.copy(tfrecord, output_example.uri+split_value+file_name, True)
                # print("file_name: "+file_name)

In [ ]:
from typing import Dict
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

"""Prepare variables"""
context=InteractiveContext()
tfrecords_dict: Dict[str, str]={
    "train":TFRECORDS_TRAIN_DATA_PATH,
    "eval":TFRECORDS_EVAL_DATA_PATH,
    "golden":TFRECORDS_GOLDEN_DATA_PATH
  }

"""Call CopyExampleGen Component"""
example_gen=tfx.components.CsvExampleGen(input_base='./data/')

context.run(example_gen)


# """Call StatisticsGen Component"""
# statistics_gen=tfx.components.StatisticsGen(examples=example_gen.outputs['examples'])

# context.run(statistics_gen)



INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.


INFO:absl:Processing input csv data ./data/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))